<a href="https://colab.research.google.com/github/pranav-nambiar/suggest-next-word/blob/main/fillintheblanks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install pytorch-transformers > /dev/null
import torch
import numpy as np
from pytorch_transformers import BertTokenizer, BertForMaskedLM
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased', output_attentions=True)
model.eval()

In [17]:
def predict():
    sentence_orig = input('Enter text:')
    if '____' not in sentence_orig:
        return sentence_orig
    
    print("Correct input format")
    
    sentence = sentence_orig.replace('____', 'MASK')
    tokens = nltk.word_tokenize(sentence)
    sentences = nltk.sent_tokenize(sentence)
    sentence = " [SEP] ".join(sentences)
    sentence = "[CLS] " + sentence + " [SEP]"
    tokenized_text = tokenizer.tokenize(sentence)
    masked_index = tokenized_text.index('mask')
    tokenized_text[masked_index] = "[MASK]"
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    segments_ids = []
    sentences = sentence.split('[SEP]')
    for i in range(len(sentences)-1):
        segments_ids.extend([i]*len(sentences[i].strip().split()))
        segments_ids.extend([i])

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
            outputs = model(tokens_tensor, token_type_ids=segments_tensors)
            predictions = outputs[0]
            attention = outputs[-1]

    dim = attention[2][0].shape[-1]*attention[2][0].shape[-1]
    a = attention[2][0].reshape(12, dim)
    b = a.mean(axis=0)
    c = b.reshape(attention[2][0].shape[-1],attention[2][0].shape[-1])
    avg_wgts = c[masked_index]

    focus = [tokenized_text[i] for i in avg_wgts.argsort().tolist()[::-1] if tokenized_text[i] not in ['[SEP]', '[CLS]', '[MASK]']][:5]

    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    
    return sentence_orig.replace('____', predicted_token)

In [21]:
predict()

Enter text:Eyes are the ____ to the world .
Correct input format


'Eyes are the key to the world .'

In [22]:
predict()

Enter text:We ____ football yesterday .
Correct input format


'We played football yesterday .'

In [25]:
predict()

Enter text:The balloon is filled with helium . so it is ____ than air .
Correct input format


'The balloon is filled with helium . so it is lighter than air .'

In [28]:
predict()

Enter text:I knew it was going to rain , but I forgot to take my ____ , and got wet on the way .
Correct input format


'I knew it was going to rain , but I forgot to take my coat , and got wet on the way .'

In [34]:
predict()

Enter text:Site activity information : We keep ____ of some of the actions you take on Facebook .
Correct input format


'Site activity information : We keep track of some of the actions you take on Facebook .'